In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    # 2개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 index 추출해서 실제 R행렬과 예측 행렬의 RMSE(오차율) 추출
    x_non_zero_i = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_i = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_i, y_non_zero_i]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_i, y_non_zero_i]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [3]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    #R = np.array(R,dtype = float)
    num_users, num_items = R.shape
    # P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    # R>0인 행 위치, 열 위치, 값을 non_zeros list 객체에 저장
    str_non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] != 0]
    for i in range(num_users):
        for j in range(num_items):
            if R[i,j] != 0:
                if R[i,j] == '매우 쉬움':
                    R[i,j] = 1
                elif R[i,j] == '쉬움':
                    R[i,j] = 2
                elif R[i,j] == '보통':
                    R[i,j] = 3
                elif R[i,j] == '어려움':
                    R[i,j] = 4
                elif R[i,j] == '매우 어려움':
                    R[i,j] = 5
    non_zeros = []
    for i, j, r in str_non_zeros:
        if r == '매우 쉬움':
            non_zeros.append((i,j,1))
        elif r == '쉬움':
            non_zeros.append((i,j,2))
        elif r == '보통':
            non_zeros.append((i,j,3))
        elif r == '어려움':
            non_zeros.append((i,j,4))
        elif r == '매우 어려움':
            non_zeros.append((i,j,5))
            
    # SGD 기법으로 P와 Q 행렬을 반복 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구하기
            eij = r - np.dot(P[i,:], Q[j,:].T)
            
            # 정규화를 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate * (eij*Q[j,:] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate * (eij*P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        
        print("### iteration step:", step, " rmse:", rmse)
        
    return P, Q

In [4]:
# 데이터 프레임 작업

review = pd.read_csv('./escapable/review.csv',encoding='CP949' ,sep = ';', names=['review_index', 'cafe_name', 'thema_name', 'user_left_time', 'user_difficulty','user_escape','user_rate','user_nickname']) # 방탈출 리뷰 파일

#review = review[['user_nickname', 'thema_name', 'user_escape']] # 유저 닉네임, 테마 이름, 평점

review_matrix = review.pivot_table('user_difficulty', index='user_nickname', columns='thema_name', aggfunc='first') # 사용자-테마 행렬

In [5]:
review

,review_index,cafe_name,thema_name,user_left_time,user_difficulty,user_escape,user_rate,user_nickname
0,1,비트포비아 미션브레이크 CGV 용산점,신서유기 : 신묘한 실종사건,300,쉬움,1,3.0,방린냥
1,2,비트포비아 미션브레이크 CGV 용산점,신서유기 : 신묘한 실종사건,2541,매우 쉬움,1,2.5,미옥
2,3,비트포비아 미션브레이크 CGV 용산점,신서유기 : 신묘한 실종사건,80,쉬움,1,2.5,진누
3,4,비트포비아 미션브레이크 CGV 용산점,신서유기 : 신묘한 실종사건,2629,매우 쉬움,1,2.0,빵수니
4,5,비트포비아 미션브레이크 CGV 용산점,신서유기 : 신묘한 실종사건,-3600,어려움,-1,3.0,튠
...,...,...,...,...,...,...,...,...
159263,161943,코드케이 홍대점,꼬레아 우라,1652,보통,1,5.0,Colory
159264,161944,코드케이 홍대점,꼬레아 우라,1650,어려움,1,4.5,깡우
159265,161945,코드케이 홍대점,꼬레아 우라,1470,보통,1,5.0,엔데
159266,161946,코드케이 홍대점,꼬레아 우라,688,보통,1,4.5,하핑


In [6]:
review_matrix

thema_name,(나쁜) 아기돼지 삼형제,13 여고괴담,170cm즈음에,1945 SPY GAME,23번째 실험연구 : 알비노,36.5 마온술사의 정원,3인의 저주,5010,501동사람들,ALBA (알바),...,화생설화 : Blooming,화이트데이,화이트룸,환생,황금 감옥 : 와캄,후레쉬망고 호스텔,휴가중,흐린날,흥보와 놀보,히로인
user_nickname,,,,,,,,,,,,,,,,,,,,,
000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123456789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,쉬움,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
힌터,NaN,NaN,NaN,NaN,NaN,NaN,쉬움,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
힌트5개쓰고탈출,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
힌트는쓰라고있는거야,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,쉬움,NaN,NaN,NaN,NaN


In [7]:
P, Q = matrix_factorization(review_matrix.values, K=50, steps=100, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step: 0  rmse: 1.8585528224231005
### iteration step: 1  rmse: 0.853375758873777
### iteration step: 2  rmse: 0.6599584895890908
### iteration step: 3  rmse: 0.6097122481499937
### iteration step: 4  rmse: 0.5866745978658242
### iteration step: 5  rmse: 0.5738699420484104
### iteration step: 6  rmse: 0.5654268019166774
### iteration step: 7  rmse: 0.5589628726561277
### iteration step: 8  rmse: 0.553448756050999
### iteration step: 9  rmse: 0.5484153022837711
### iteration step: 10  rmse: 0.5436136417197385
### iteration step: 11  rmse: 0.5388685903980095
### iteration step: 12  rmse: 0.5340306131317224
### iteration step: 13  rmse: 0.5289689471946624
### iteration step: 14  rmse: 0.5235744484442657
### iteration step: 15  rmse: 0.517763298126907
### iteration step: 16  rmse: 0.5114815503686228
### iteration step: 17  rmse: 0.5047099891206502
### iteration step: 18  rmse: 0.4974668577689342
### iteration step: 19  rmse: 0.48980619068898684
### iteration step: 20  rmse: 0.

In [8]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=review_matrix.index, columns=review_matrix.columns)
ratings_pred_matrix.head(5)

thema_name,(나쁜) 아기돼지 삼형제,13 여고괴담,170cm즈음에,1945 SPY GAME,23번째 실험연구 : 알비노,36.5 마온술사의 정원,3인의 저주,5010,501동사람들,ALBA (알바),...,화생설화 : Blooming,화이트데이,화이트룸,환생,황금 감옥 : 와캄,후레쉬망고 호스텔,휴가중,흐린날,흥보와 놀보,히로인
user_nickname,,,,,,,,,,,,,,,,,,,,,
000,2.530263,2.049446,3.172735,3.143649,3.203969,2.626316,2.899226,3.677722,3.504598,4.312808,...,3.270709,2.704469,4.958177,2.679273,4.449562,2.207433,4.001311,3.152446,2.393614,2.605641
002,1.877465,2.306768,2.760415,2.474880,2.989923,2.774313,2.682982,3.564727,3.103914,3.710401,...,2.395255,2.262258,3.949076,2.728188,2.716830,1.844106,2.617590,2.401260,2.159277,2.594400
100,2.223682,2.388237,2.816535,2.561924,3.068773,2.479835,2.921344,3.558437,2.774655,3.417387,...,2.207593,2.389193,3.785401,2.294840,3.100927,2.053999,2.893279,2.519491,2.072554,2.439639
11,2.618545,2.392416,3.132917,3.123417,3.382732,2.753872,3.223255,4.181935,3.337009,4.039820,...,2.895220,2.858331,4.947991,2.849127,3.806305,2.427934,3.770779,3.275327,2.492497,2.990168
123456789,2.372937,1.851482,2.886024,2.670862,3.312753,2.801968,2.908195,3.846124,3.072331,3.497172,...,2.070532,2.143478,3.961553,2.340109,2.777976,1.902989,3.341335,2.791952,2.254904,2.390416


In [9]:
ratings_pred_matrix[ratings_pred_matrix.index=='딸기']
for value in ratings_pred_matrix[ratings_pred_matrix.index=='딸기'].values:
    print(value)

[2.90590922 2.17902799 3.07218052 2.90910587 2.40080627 1.7923971
 2.26870826 3.58164939 2.46299639 3.52283007 2.14983553 1.81074066
 3.14851816 3.06703236 2.87490768 3.09637532 2.44225254 3.23387626
 2.60388135 2.87671793 2.69662614 2.2042687  3.42760119 3.19482378
 3.08210343 4.23212181 3.04755726 1.98383729 3.74573709 3.07384142
 3.22942961 3.9149638  2.25371707 2.76335102 3.51214645 2.01993099
 2.63920851 3.41732294 2.93755496 3.26652255 3.41824113 2.20446917
 3.1262197  3.43953462 2.37896141 2.27740581 3.17717664 3.24770704
 1.78238616 2.33510394 2.87630787 3.97078007 3.11953049 3.71837272
 2.40637112 2.61359047 3.83705695 2.06368648 3.48653939 3.00067353
 2.52085939 3.28611628 2.11645974 3.47865302 3.54304483 4.26569693
 3.48594028 2.48104728 2.56573888 3.15636    1.44830377 1.76037114
 2.66661102 3.05041533 3.09703128 2.79448557 2.11358074 2.3678078
 2.40579076 2.52302647 2.30281436 2.52924232 2.34387205 1.18587241
 3.74394153 3.0558731  3.54910076 2.60959478 2.43598108 2.534545

In [12]:
R = review_matrix.values
R = np.array(R,dtype = float)
num_users, num_items = review_matrix.values.shape
al = 0
cnt = 0
np.nan_to_num(R,copy=False)
str_non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] != 0]
for i,j,r in str_non_zeros:
    if abs(ratings_pred_matrix.values[i,j] - r) < 0.5:
        cnt += 1
    al += 1
print("학습 성공률 : " + str(float(cnt/al)))

학습 성공률 : 0.9034261633144146
